# Semantic Kernel with OpenBnB MCP Server Integration

This notebook demonstrates how to use Semantic Kernel with the actual OpenBnB MCP server to search for real Airbnb accommodations using MCPStdioPlugin. For LLM Access, its uses Azure AI Foundry. To setup your environment variables, you can follow the [Setup Lesson ](/00-course-setup/README.md)

## Import the Needed Packages

In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Creating the MCP Plugin Connection

We'll connect to the [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) using MCPStdioPlugin. This server provides Airbnb search functionality through the @openbnb/mcp-server-airbnb package.

## Creating the Client

In this sample, we will use Azure AI Foundry for our LLM access. Make sure your environment variables are set up correctly.

## Environment Configuration

Configure Azure OpenAI settings. Make sure you have the following environment variables set:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`

In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Understanding the OpenBnB MCP Integration

This notebook connects to the **real OpenBnB MCP server** that provides actual Airbnb search functionality.

### How it works:

1. **MCPStdioPlugin**: Uses standard input/output communication with the MCP server
2. **Real NPM Package**: Downloads and runs `@openbnb/mcp-server-airbnb` via npx
3. **Live Data**: Returns actual Airbnb property data from their APIs
4. **Function Discovery**: The agent automatically discovers available functions from the MCP server

### Available Functions:

The OpenBnB MCP server typically provides:
- **search_listings** - Search for Airbnb properties by location and criteria
- **get_listing_details** - Get detailed information about specific properties
- **check_availability** - Check availability for specific dates
- **get_reviews** - Retrieve reviews for properties
- **get_host_info** - Get information about property hosts

### Prerequisites:

- **Node.js** installed on your system
- **Internet connection** to download the MCP server package
- **NPX** available (comes with Node.js)

### Testing the Connection:

You can test the MCP server manually by running:
```bash
npx -y @openbnb/mcp-server-airbnb
```

This will download and start the OpenBnB MCP server, which Semantic Kernel then connects to for real Airbnb data.

## Running the Agent with OpenBnB MCP Server

Now we will run the AI Agent that connects to the OpenBnB MCP server to search for real Airbnb accommodations in Stockholm for 2 adults and 1 kid. Feel free to change the `user_inputs` list to modify the search criteria.

In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Summary
Congratulations! You've successfully built an AI agent that integrates with real-world accommodation search using the Model Context Protocol (MCP):

## Technologies Used:
- Semantic Kernel - For building intelligent agents with Azure OpenAI
- Azure AI Foundry - For LLM capabilities and chat completion
- MCP (Model Context Protocol) - For standardized tool integration
- OpenBnB MCP Server - For real Airbnb search functionality
- Node.js/NPX - For running the external MCP server

## What You've Learned:
- MCP Integration: Connecting Semantic Kernel agents to external MCP servers
- Real-time Data Access: Searching actual Airbnb properties through live APIs
- Protocol Communication: Using stdio communication between agent and MCP server
- Function Discovery: Automatically discovering available functions from MCP servers
- Streaming Responses: Capturing and logging function calls in real-time
- HTML Rendering: Formatting agent responses with styled tables and interactive displays

## Next Steps:
- Integrate additional MCP servers (weather, flights, restaurants)
- Build a multi-agent system combining MCP and A2A protocols
- Create custom MCP servers for your own data sources
- Implement persistent conversation memory across sessions
- Deploy the agent to Azure Functions with MCP server orchestration
- Add user authentication and booking capabilities
